# Regression with Gaussian Processes

------------------------------------------------------
*Machine Learning, Master in Big Data Analytics, 2020-2021*

*Pablo M. Olmos olmos@tsc.uc3m.es, Emilio Parrado Hernandez, emilio.parrado@uc3m.es*


---


José María Martínez Marín 100443343

Azamat Ziiatdinov 100460540

------------------------------------------------------

The aim of this homework is to solve a real data problem using the Gaussian Process implementation of GPy. The documentation of GPy is avaialable from the [SheffieldML github page](https://github.com/SheffieldML/GPy) or from [this page](http://gpy.readthedocs.org/en/latest/). 

The problem is the prediction of both the heating load (HL) and cooling load (CL) of residential buildings. We consider eight input variables for each building: relative compactness, surface area, wall area, roof area, overall height, orientation, glazing area, glazing area distribution.

In this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) you can find a detailed description of the problem and a solution based on linear regression [(iteratively reweighted least squares (IRLS) algorithm)](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwjZuoLY2OjgAhUs3uAKHUZ7BVcQFjAJegQIAhAC&url=https%3A%2F%2Fpdfs.semanticscholar.org%2F9b92%2F18e7233f4d0b491e1582c893c9a099470a73.pdf&usg=AOvVaw3YDwqZh1xyF626VqfnCM2k) and random forests. Using GPs, our goal is not only estimate accurately both HL and CL, but also get a measure of uncertainty in our predictions.

The data set can be downloaded from the [UCI repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency#).

In [ ]:
!pip install GPy 

     |████████████████████████████████| 1.0MB 8.2MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
  Created wheel for GPy: filename=GPy-1.9.9-cp37-cp37m-linux_x86_64.whl size=2626993 sha256=18b325f92159a980b6ff85f1d702439744ba3a072ad5093265e50b08c73a6a7d
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp37-none-any.whl size=102552 sha256=0dc477f13d12ad8c220fd3b90b33c23ba7d9587caa6dfc3eb69f323531fa7784
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPy paramz


### 1. Loading and preparing the data

* Download the dataset
* Divide at random the dataset into train (80%) and test (20%) datasets 

In [ ]:
from google.colab import files

uploaded = files.upload()



Saving ENB2012_data.xlsx to ENB2012_data.xlsx


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import GPy
import numpy as np
import matplotlib.pyplot as plt

nomlist_train = 'ENB2012_data.xlsx';

df = pd.read_excel (nomlist_train)
dftrain, dftest = train_test_split(df,test_size=0.2)



In [ ]:
HLy_train = dftrain["Y1"]
CLy_train = dftrain["Y2"]
Ytrain = dftrain.iloc[:, 8:10].values
Ytrain1 = HLy_train.values
Ytrain2 = CLy_train.values
HLy_test = dftest["Y1"]
CLy_test = dftest["Y2"]
Ytest1 = HLy_test.values
Ytest2 = CLy_test.values


In [ ]:
Xtrain = dftrain[dftrain.columns[:8]].values
Xtest = dftest[dftest.columns[:8]].values
X = df.iloc[:, 0:6].values

### 2. Setting and optimizing the model

You will train two independent GPs, one to estimate HL and one to estimate CL. For each of the two GPs ...

**On the training data set:**

a) Build a GP regression model based on a RBF kernel with ARD, in which each input dimension is weighted with a different lengthscale. **2.5 points**

b) Fit the covariance function parameters and noise variance. **1 point** 

c) According to the ARD parameters found, what variables are more important for the regression? Compare it to Table 8 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X) **2.5 points**

**On the test data set:**

d) Compute the test mean absolute error error and the test mean square error (MSE)  using the GP posterior mean and the optimized hyperparameters. Compare your results with Tables 6 and 7 in this [paper](https://www.sciencedirect.com/science/article/pii/S037877881200151X).
**2 points**

e) Try to improve your results by using a more complicated kernel, in which you combine various covariance functions. In this [link](http://nbviewer.jupyter.org/github/SheffieldML/notebook/blob/master/GPy/basic_kernels.ipynb) you can see how to define different kernels and combine  them. Comments the results. **2 points**





---



**a) Build a GP regression model based on a RBF kernel with ARD, in which each input dimension is weighted with a different lengthscale.**

**b) Fit the covariance function parameters and noise variance.**

In [ ]:

#HL

ker = GPy.kern.RBF(8,ARD=True)
m = GPy.models.GPRegression(dftrain[dftrain.columns[:8]].values, dftrain['Y1'].values.reshape(-1,1), ker)
m.optimize(messages=True,max_f_eval = 1000)

print(ker.parameter_names())
print(ker.variance.values)
print(ker.lengthscale.values)
print(m.Gaussian_noise.variance.values)

['variance', 'lengthscale']
[479.0536375]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 2.03099936e+02 5.13884397e-06 6.96230208e+01]
[0.19559428]


In [ ]:
m.optimize_restarts(num_restarts = 10)

Optimization restart 1/10, f = 663.908897982158
Optimization restart 2/10, f = 879.9053516919315
Optimization restart 3/10, f = 879.9053327682822
Optimization restart 4/10, f = 700.9918764229565
Optimization restart 5/10, f = 605.1083496182845
Optimization restart 6/10, f = 1191.9936593012305
Optimization restart 7/10, f = 647.2367921794546
Optimization restart 8/10, f = 647.9133709121124
Optimization restart 9/10, f = 605.4622103138331
Optimization restart 10/10, f = 683.885725730195


In [ ]:
#CL
ker2 = GPy.kern.RBF(8,ARD=True)
m2 = GPy.models.GPRegression(dftrain[dftrain.columns[:8]].values, dftrain['Y2'].values.reshape(-1,1), ker2)
m2.optimize(messages=True,max_f_eval = 1000)

print(ker2.parameter_names())
print(ker2.variance.values)
print(ker2.lengthscale.values)
print(m2.Gaussian_noise.variance.values)

['variance', 'lengthscale']
[511.4810082]
[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.26749193e+05 1.22562177e+00 1.48021579e+05]
[2.89112539]


In [ ]:
m2.optimize_restarts(num_restarts = 10)

 /usr/local/lib/python3.7/dist-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/10, f = 1281.8274165118835
Optimization restart 2/10, f = 1281.827468830673
Optimization restart 3/10, f = 1281.827417235995
Optimization restart 4/10, f = 1281.827436999863
Optimization restart 5/10, f = 1281.8275276185036
Optimization restart 6/10, f = 1281.8275213815464
Optimization restart 7/10, f = 1281.8275918790728
Optimization restart 8/10, f = 1281.8274738936198
Optimization restart 9/10, f = 1296.7927106948064
Optimization restart 10/10, f = 1530.1837601781485


**c) According to the ARD parameters found, what variables are more important for the regression? Compare it to Table 8 in this paper**

---

The most important variables for the regression are the last three of them, X6, X7 and X8, with $10^5$ times higher. For HL, the most important variable is X8, but for CL, both X6, X7 and X8 have similar weights, thus having similar importance. Compared with Table 8 from the paper, we can see that, for HL, the most important variable is X7 by far (X1 and X2 are important too) and, for CL, same thing. The least important variable is X5, so it can be concluded that the results given in this project are similar only in what respects to the importance of X7.

**d) Compute the test mean absolute error error and the test mean square error (MSE) using the GP posterior mean and the optimized hyperparameters. Compare your results with Tables 6 and 7 in this paper.**


---




In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#HL

posteriorYtest = m.posterior_samples_f(Xtrain, full_cov=True, size=10)
meanYtest = m.predict(Xtest,full_cov=True)

MAE1 = mean_absolute_error(HLy_test, meanYtest[0])
MSE1 = mean_squared_error(HLy_test, meanYtest[0])

print(MAE1)
print(MSE1)

0.31061983789804665
0.17741780666442214


 /usr/local/lib/python3.7/dist-packages/GPy/core/gp.py:598: RuntimeWarning:covariance is not positive-semidefinite.


In [ ]:
#CL

posteriorYtest2 = m2.posterior_samples_f(Xtrain, full_cov=True, size=10)
meanYtest2 = m2.predict(Xtest,full_cov=True)

MAE2 = mean_absolute_error(CLy_test, meanYtest2[0])
MSE2 = mean_squared_error(CLy_test, meanYtest2[0])

print(MAE2)
print(MSE2)

1.0222030899433758
2.255547319804176


 /usr/local/lib/python3.7/dist-packages/GPy/core/gp.py:598: RuntimeWarning:covariance is not positive-semidefinite.


It can be seen that both Mean Absolute Error and Mean Squared Errors are much smaller for HL, for which **MAE = 0.31** and **MSE = 0.18** than for CL, for which **MAE = 1.02** and **MSE = 2.26** (up to 20 times higher for MSE). 

On the other hand, comparing this results to the ones in tables 5 and 6 of the paper, it can be seen that, for the Random Forest model,, which is the best model in the paper, both MAE are slightly higher (a 30-40%), and MSE for HL is almost the same, but for CL, it is three times higher.

Regarding the worst model, IRLS, the results are improved even more, specially for HL, up to an 85% lower regarding MAE, and a 92% regarding MSE.

**e) Try to improve your results by using a more complicated kernel, in which you combine various covariance functions. In this link you can see how to define different kernels and combine them. Comments the results.**


---

For this task, an exponentiated quadratic kernel has been multiplied by the RBF one used before:

$$ k(r) = \sigma ^2 (1+\sqrt 5 r + \frac{5}{3} r^2)e^{-\sqrt 5 r}$$

This kernel is similar to the RBF one, however, it doesn't have features for doing variational kernels (the so-called psi-statistics).

In [ ]:
#HL
ker = GPy.kern.RBF(8,ARD=True)
ker12 = GPy.kern.ExpQuad(input_dim=1, lengthscale=3, active_dims=[1]) 
k = ker * ker12 

m = GPy.models.GPRegression(dftrain[dftrain.columns[:8]].values, dftrain['Y1'].values.reshape(-1,1), k)
m.optimize(messages=True,max_f_eval = 1000)


In [ ]:
#CL

ker2 = GPy.kern.RBF(8,ARD=True)
ker22 = GPy.kern.ExpQuad(input_dim=1, lengthscale=3, active_dims=[1]) 
k = ker2 * ker22
m2 = GPy.models.GPRegression(dftrain[dftrain.columns[:8]].values, dftrain['Y2'].values.reshape(-1,1), k)
m2.optimize(messages=True,max_f_eval = 1000)



In [ ]:
#HL

posteriorYtest3 = m.posterior_samples_f(Xtrain, full_cov=True, size=10)
meanYtest3 = m.predict(Xtest,full_cov=True)

MAE3 = mean_absolute_error(HLy_test, meanYtest3[0])
MSE3 = mean_squared_error(HLy_test, meanYtest3[0])

print(MAE3)
print(MSE3)

0.33635422277215543
0.24757836322467536


 /usr/local/lib/python3.7/dist-packages/GPy/core/gp.py:598: RuntimeWarning:covariance is not positive-semidefinite.


In [ ]:
#CL

posteriorYtest4 = m2.posterior_samples_f(Xtrain, full_cov=True, size=10)
meanYtest4 = m2.predict(Xtest,full_cov=True)

MAE4 = mean_absolute_error(CLy_test, meanYtest4[0])
MSE4 = mean_squared_error(CLy_test, meanYtest4[0])

print(MAE4)
print(MSE4)

0.4817678970650989
0.5109196959464339


From the results above, it can be seen that both MAE and MSE are slightly greater for HL (**MAE = 0.34** and **MSE = 0.25**), up to a 70% gap for MSE, nonetheless, for CL the results are clearly improved (**MAE = 0.48** and **MSE = 0.51**), up to an 80% lower for MSE and a 50% lower for MAE.

It can be stated that the shape of the data for CL fits better that exponenciated quadratic kernel, and as for HL, perhaps a linear or quadratic model could improve those results.